In [ ]:
import ibalgo

In [ ]:
def rsi_long(online_indicator, rsi_threshold=35,):
    indicators =  online_indicator.temp_indicator
    condition = (indicators.rsi < rsi_threshold) # and (indicators.low < 3345)
    return condition

def sto_long_w_price_point(online_indicator, sto_threshold=20, sto_range_thresh=8, price_point=3360):
    indicators =  online_indicator.temp_indicator
    condition = (indicators.sto < sto_threshold) and ((indicators.sto_high - indicators.sto_low) > sto_range_thresh) and (indicators.close <= price_point)
    return condition

def sto_long_at_night(online_indicator, sto_threshold=20, sto_range_thresh=35, rsi_thresh=55):
    indicators =  online_indicator.temp_indicator
    condition = (indicators.sto < sto_threshold) and ((indicators.sto_high - indicators.sto_low) >= sto_range_thresh) and (indicators.rsi < rsi_thresh)
    return condition


In [ ]:
entry_sto_long_buy = ibalgo.Entry(sto_long_at_night, entry_price_offset=0, action='BUY',
                                  stop_loss=10.00, profit_taker=20.00, trade_wait_tm=15,
                                  quantity=2)

In [ ]:
#online_indicator = ibalgo.OnlineIndicator(sto_period=60) # daytime (higher frequency trades)
online_indicator = ibalgo.OnlineIndicator(sto_period=(60 * 24), rsi_period=25) # nightime

In [ ]:
entry_obj_dict = {'sto_long': entry_sto_long_buy}

In [ ]:
app = ibalgo.App(
    entry_objs=entry_obj_dict,
    online_indicator=online_indicator,
    show_indicators=['open', 'high', 'low', 'close', 'volume',
                     'rsi', 'adx', 'mean_pr', 'sma_pr',
                     'ewa_pr', 'pr_diff_ewa', 'volume_roc', 'sto',
                     'sto_high', 'sto_low'],
    trade_mode=True, trade_and_disconnect=False,
    trade_loss_disconnect=True, loss_limit=1,
    loss_timeout=1
)

In [ ]:
app.connect('127.0.0.1',  7497, clientId=0)

In [ ]:
appl_req_ids = app.get_historical_and_trade_strategy_futs(
    'MES', close_app=True, only_RTH=0, history_len=20,  bar_length=1, bar_unit='min',
    secType='FUT', exchange='GLOBEX', trade_strategy_cnt_thresh=10,
    transmit=True)

In [ ]:
app.run()

In [ ]:
app.historical_indicators_df.to_parquet(f'{app.logging_dir}/streaming_data.parquet')